In [ ]:
# conda activate tf_gpu 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import matplotlib
import importlib
from datapaths import *
import affective_utils as au

%matplotlib inline

In [2]:
labelfile = './CowenKeltnerEmotionalVideos.csv'
labeldf = pd.read_csv(labelfile)

In [3]:
labeldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2185 entries, 0 to 2184
Columns: 649 entries, Filename to zeal
dtypes: float64(595), int64(53), object(1)
memory usage: 10.8+ MB


In [4]:
labeldf.head()

,Filename,Admiration,Adoration,Aesthetic Appreciation,Amusement,Anger,Anxiety,Awe,Awkwardness,Boredom,...,vitality,vulnerability,warmth,weakness,weariness,wonder,worry,wrath,yearning,zeal
0,0001.mp4,0.000000,0.0,0.083333,0.00000,0.083333,0.083333,0.083333,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0
1,0002.mp4,0.000000,0.0,0.000000,0.25000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0
2,0003.mp4,0.083333,0.0,0.083333,0.66667,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0
3,0004.mp4,0.000000,0.0,0.000000,0.75000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.0,0.0,0.0,0.0,0.11111,0.0,0.00000,0.0,0.0,0.0
4,0005.mp4,0.000000,0.0,0.000000,0.00000,0.000000,0.333330,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.11111,0.0,0.0,0.0


In [5]:
emocols= labeldf.columns[list(range(0,34))]
affectdimcols= labeldf.columns[list(range(34,48))]
freewordcols=labeldf.columns[list(range(48,648))]

In [6]:
sampletrain= labeldf.head(99).iloc[:89][emocols]
sampletest = labeldf.head(99).iloc[89:][emocols]

In [7]:
sampletrain.tail() 

,Filename,Admiration,Adoration,Aesthetic Appreciation,Amusement,Anger,Anxiety,Awe,Awkwardness,Boredom,...,Joy,Nostalgia,Pride,Relief,Romance,Sadness,Satisfaction,Sexual Desire,Surprise,Sympathy
84,0085.mp4,0.11111,0.00000,0.0,0.22222,0.0,0.00000,0.11111,0.0,0.11111,...,0.111110,0.0,0.0,0.00000,0.0,0.00000,0.22222,0.0,0.00000,0.00000
85,0086.mp4,0.00000,0.11111,0.0,0.44444,0.0,0.11111,0.11111,0.0,0.11111,...,0.055556,0.0,0.0,0.11111,0.0,0.00000,0.11111,0.0,0.11111,0.00000
86,0087.mp4,0.00000,0.22222,0.0,0.55556,0.0,0.00000,0.11111,0.0,0.00000,...,0.555560,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.22222,0.00000
87,0088.mp4,0.11111,0.00000,0.0,0.55556,0.0,0.00000,0.00000,0.0,0.00000,...,0.000000,0.0,0.0,0.00000,0.0,0.00000,0.22222,0.0,0.11111,0.22222
88,0089.mp4,0.00000,0.00000,0.0,0.11111,0.0,0.00000,0.00000,0.0,0.00000,...,0.000000,0.0,0.0,0.00000,0.0,0.22222,0.00000,0.0,0.11111,0.00000


In [8]:
sampletest.tail()

,Filename,Admiration,Adoration,Aesthetic Appreciation,Amusement,Anger,Anxiety,Awe,Awkwardness,Boredom,...,Joy,Nostalgia,Pride,Relief,Romance,Sadness,Satisfaction,Sexual Desire,Surprise,Sympathy
94,0095.mp4,0.0,0.076923,0.0,0.076923,0.076923,0.076923,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.076923,0.153850
95,0096.mp4,0.0,0.000000,0.0,0.000000,0.000000,0.538460,0.000000,0.15385,0.076923,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
96,0097.mp4,0.0,0.000000,0.0,0.000000,0.000000,0.615380,0.000000,0.00000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.076923
97,0098.mp4,0.0,0.538460,0.0,0.384620,0.076923,0.000000,0.153850,0.00000,0.076923,...,0.076923,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.000000,0.000000
98,0099.mp4,0.0,0.000000,0.0,0.307690,0.000000,0.000000,0.076923,0.00000,0.000000,...,0.230770,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.230770,0.076923


In [9]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd

from skimage.transform import resize

from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.
/home/yt/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yt/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yt/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yt/anaconda3/envs/tf_gpu/l

In [10]:
# open the .txt file which have names of training videos
f = open("traintestsplit/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,0001.mp4
1,0002.mp4
2,0003.mp4
3,0004.mp4
4,0005.mp4


In [11]:
# open the .txt file which have names of test videos
f = open("traintestsplit/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,0090.mp4
1,0091.mp4
2,0092.mp4
3,0093.mp4
4,0094.mp4


In [12]:
sampletrainindexed = sampletrain.set_index('Filename')
sampletestindexed = sampletest.set_index('Filename')

In [13]:
sampletrainindexed.values.shape , sampletestindexed.shape

((89, 33), (10, 33))

In [78]:
def extract_images(videoFile):
    cap = cv2.VideoCapture('Videos/'+videoFile) #.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(1) #frame rate
    count = 0
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        '''
        if (frameId % math.floor(frameRate) == 0): 
        '''
        # storing the frames in a new folder named train_1
        filename ='train_1/' + videoFile +"_frame%d.jpg" % count
        count+=1
        cv2.imwrite(filename, frame)
        
    cap.release()
    
    return count

In [124]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    tag = sampletrainindexed.loc[train['video_name'][i]].idxmax()
    train_video_tag.append(tag)
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    tag = sampletestindexed.loc[test['video_name'][i]].idxmax()
    test_video_tag.append(tag)
    
test['tag'] = test_video_tag

In [125]:
train.head()

,video_name,frame_count,tag
0,0001.mp4,47,Interest
1,0002.mp4,64,Nostalgia
2,0003.mp4,120,Amusement
3,0004.mp4,25,Amusement
4,0005.mp4,151,Confusion


In [80]:
# storing the frames from training videos
train_frames=[]
for i in tqdm(range(train.shape[0])):
    videoFile = train['video_name'][i]
    frame_count = extract_images(videoFile)
    train_frames.append( frame_count)
    print(videoFile+' frames %d' % frame_count)

train['frame_count'] = train_frames

  1%|          | 1/89 [00:00<00:26,  3.33it/s]

0001.mp4 frames 47
0002.mp4 frames 64


  4%|▍         | 4/89 [00:00<00:17,  4.85it/s]

0003.mp4 frames 120
0004.mp4 frames 25


  6%|▌         | 5/89 [00:01<00:27,  3.09it/s]

...

0086.mp4 frames 377
0087.mp4 frames 90


 99%|█████████▉| 88/89 [00:40<00:00,  3.63it/s]

0088.mp4 frames 60


100%|██████████| 89/89 [00:41<00:00,  2.15it/s]

0089.mp4 frames 61


In [44]:
train_multiclass = pd.DataFrame()

In [34]:
df0 = pd.DataFrame(sampletrainindexed.loc[train['video_name'][0]].values).transpose()
df1 = pd.DataFrame(sampletrainindexed.loc[train['video_name'][1]].values).transpose()

In [35]:
df0.append(df1)

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.0,0.0,0.083333,0.00,0.083333,0.083333,0.083333,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.16667,0.0,0.00,0.25,0.0
0,0.0,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.083333,0.0,...,0.0,0.5,0.0,0.0,0.083333,0.00000,0.0,0.25,0.00,0.0


In [45]:
# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
train_multiclass = pd.DataFrame()
for i in tqdm(range(len(images))):
    filename = images[i].split('/')[1]
    # creating the image name
    train_image.append(filename)
    # creating the class of image
    classname= sampletrainindexed.loc[filename.split('_')[0]].idxmax()
    train_class.append(classname)
    # creating multiclass label
    video_label_df = pd.DataFrame(sampletrainindexed.loc[filename.split('_')[0]].values).transpose()
    train_multiclass=train_multiclass.append(video_label_df)
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('Cowen/train_new.csv',header=True, index=False)
train_multiclass.to_csv('Cowen/train_y.csv',header=True, index=False)

100%|██████████| 11646/11646 [00:13<00:00, 886.93it/s]


In [46]:
train_multiclass

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.0,0.0,0.000000,0.58333,0.000000,0.000000,0.000000,0.16667,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.083333,0.00000,0.0,0.25000,0.41667
0,0.3,0.0,0.100000,0.40000,0.000000,0.000000,0.300000,0.10000,0.100000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.20000,0.0,0.10000,0.00000
0,0.3,0.0,0.100000,0.40000,0.000000,0.000000,0.300000,0.10000,0.100000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.20000,0.0,0.10000,0.00000
0,0.0,0.0,0.400000,0.10000,0.000000,0.000000,0.200000,0.00000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.10000,0.00000
0,0.0,0.0,0.000000,0.30000,0.000000,0.000000,0.000000,0.10000,0.000000,0.00000,...,0.1,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.10000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,0.555560,0.11111,0.000000,0.000000,0.000000,0.00000,0.055556,0.22222,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.11111,0.0,0.22222,0.00000
0,0.0,0.0,0.083333,0.00000,0.083333,0.083333,0.083333,0.00000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.166670,0.00000,0.0,0.25000,0.00000
0,0.0,0.0,0.000000,0.30000,0.000000,0.000000,0.000000,0.10000,0.000000,0.00000,...,0.1,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.10000,0.00000
0,0.0,0.0,0.555560,0.11111,0.000000,0.000000,0.000000,0.00000,0.055556,0.22222,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.11111,0.0,0.22222,0.00000


In [43]:
train_data

,image,class
0,0027.mp4_frame72.jpg,Amusement
1,0032.mp4_frame190.jpg,Amusement
2,0032.mp4_frame397.jpg,Amusement
3,0042.mp4_frame64.jpg,Aesthetic Appreciation
4,0053.mp4_frame729.jpg,Disgust
...,...,...
11641,0061.mp4_frame120.jpg,Aesthetic Appreciation
11642,0001.mp4_frame21.jpg,Interest
11643,0053.mp4_frame156.jpg,Disgust
11644,0061.mp4_frame48.jpg,Aesthetic Appreciation
